In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import seaborn as sns
import inflection
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_raw = pd.read_csv('./data/raw/Ecommerce.csv',encoding='unicode_escape')

In [ ]:
df_raw.drop('Unnamed: 8',axis=1, inplace=True)

In [ ]:
df_raw.head()

In [ ]:
cols = [ inflection.underscore(col)  for col in df_raw.columns]
df_raw.columns = cols

In [ ]:
df_raw.shape

In [ ]:
df_raw.info()

### Replace NA

In [ ]:
df_raw.isna().sum()

In [ ]:
df_raw.dropna(axis=0, inplace=True)

In [ ]:
df_raw.shape

### Change Data types

In [ ]:
# invoice date
df_raw['invoice_date'] = pd.to_datetime( df_raw['invoice_date'], format='%d-%b-%y' )

# customer id
df_raw['customer_id'] = df_raw['customer_id'].astype( int )
df_raw.head()

### Feature Engineering

In [ ]:
# data reference
df_ref = pd.DataFrame(df_raw['customer_id'].unique(),columns=['customer_id'])
df_ref.head()

In [ ]:
# gross revenue (faturamento) = quantity * price 
df_raw['gross_revenue'] = df_raw['quantity'] * df_raw['unit_price']

In [ ]:
# Monetary
df_monetary = df_raw[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )

In [ ]:
df_ref.head()

In [ ]:
# Recency - Last day purchase
# pegando a data mais recente de compra para cada customer
df_recency = df_raw[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
# calculando o recency_days = data máxima do dataset - data máxima de compra de cada customer
df_recency['recency_days'] = ( df_raw['invoice_date'].max() - df_recency['invoice_date'] ).dt.days

In [ ]:
df_recency.head(3)

In [ ]:
# criando uma cópia sem a coluna invoice_date
df_recency = df_recency[['customer_id', 'recency_days']].copy()
# left join da tabela df_ref com df_recency
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

In [ ]:
df_ref.head(3)

In [ ]:
# Frequency
df_freq = df_raw[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )

In [ ]:
df_ref.head(3)

### Hyperparameter Fine-Tunning

In [ ]:
X = df_ref.drop(columns=['customer_id'])
X.head()

A primeira métrica de clusterização que iremos utilizar é a Within Cluster Sum of Square (WSS). Primeiro definimos os números de clusters possíveis para o dataset, um mero chute. E para cada númeo de clusters olharemos a métrica WSS.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
clusters = [2,3,4,5,6]

In [ ]:
wss = []
for k in clusters:
    # model definition
    kmeans = KMeans(init='random',n_clusters=k,n_init=10, max_iter=300,random_state=42)
    # model training
    kmeans.fit(X)
    # validation
    wss.append(kmeans.inertia_)

In [ ]:
# valor de wss para cada k
wss

Elbow method

In [ ]:
sns.set_theme()

In [ ]:
# plt.plot(clusters,wss, linestyle='--',marker='o', color='b')
# plt.xlabel('K')
# plt.ylabel('Within-Cluster Sum of Square')
# plt.title('WSS vs K')
# plt.show()

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
kmeans = KElbowVisualizer(KMeans(), k=clusters, timings=False)
kmeans.fit(X)
kmeans.show()

In [ ]:
kmeans = KElbowVisualizer(KMeans(), k=clusters, metric='silhouette',timings=False)
kmeans.fit(X)
kmeans.show()

### Model Training

In [43]:
k=3
# model definition
kmeans = KMeans(init='random',n_clusters=k, n_init=10, max_iter=300, random_state=42)
# model training
kmeans.fit(X)
# clustering
labels = kmeans.labels_

### Cluster Analysis

In [44]:
from sklearn.metrics import silhouette_score

In [45]:
wss = kmeans.inertia_
ss = silhouette_score(X, labels, metric='euclidean')

In [46]:
# WSS
print(f'WSS value:{wss}')
print(f'Silhouette score:{ss}')

WSS value:52830442008.00697
Silhouette score:0.9494677566681542


In [48]:
df_ref['cluster'] = labels
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_no,cluster
0,17850,5288.63,302,35,0
1,13047,3079.10,31,18,0
2,12583,7187.34,2,18,0
3,13748,948.25,95,5,0
4,15100,635.10,330,6,0


In [50]:
fig = px.scatter_3d(df_ref, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster')
fig.show()

### Cluster Profile

Vamos avaliar as seguintes métricas para definir os perfis dos clusters:

- Número e percentual de clientes em cada
- Média de recência em cada
- Média de gross revenue em cada
- Média de invoice no em cada

In [65]:
df_cluster = df_ref.groupby("cluster", as_index=False).agg(
    {
        "customer_id": "count",
        "gross_revenue": "mean",
        "recency_days": "mean",
        "invoice_no": "mean",
    }
)
df_cluster["perc_customer"] = 100 * (df_cluster["customer_id"] / df_cluster["customer_id"].sum())
df_cluster

,cluster,customer_id,gross_revenue,recency_days,invoice_no,perc_customer
0,0,4335,1372.577826,92.257901,4.614533,99.153705
1,1,6,182181.981667,7.166667,89.000000,0.137237
2,2,31,40543.517742,13.290323,53.290323,0.709058


#### Cluster 01:  ( Candidato à Insider )
    - Número de customers: 6 (0.14% do customers )
    - Recência em média: 7 dias
    - Compras em média: 89 compras
    - Receita em média: $182.182,00 dólares
        
#### Cluster 02: 
    - Número de customers: 31 (0.71 do customers )
    - Recência em média: 14 dias
    - Compras em média: 53 compras
    - Receita em média: $40.543,52 dólares
        
#### Cluster 03: 
    - Número de customers: 4.335 (99% do customers )
    - Recência em média: 92 dias
    - Compras em média: 5 compras
    - Receita em média: $1.372,57 dólares